In [1]:
import os
from Bio import SeqIO
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [3]:
constructs = pd.read_csv('datasets/constructs.csv')
constructs

,id,short_name,full_name
0,A1,e11x32M86,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP1-P(Cym)-32-...
1,A2,e11x30STNpuSspS2,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP2-P(Cym)-30-...
2,A3,e11x32NpuSspS2,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP2-P(Cym)-32-...
3,A4,e11x30NpuSspS1,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP2-P(Cym)-30-...
4,A5,e11x32gp411,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP3-P(Cym)-32-...
...,...,...,...
360,3K-E422,gfp_med,NaN
361,E422,gfp_low,NaN
362,C+,positive_control,NaN
363,C-,negative_control,NaN


In [4]:
def parse_minutes(x):
    
    spl = x.split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

def read_plate_data(df):
    
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.dropna(inplace=True)
    return df.reset_index(drop=True)

def transpose_data(df):
    
    df.set_index('short_name', inplace=True)
    df = df.transpose().reset_index()
    df['time'] = df['index'].apply(parse_minutes)
    df = df.set_index('time')
    df = df.drop('index', axis=1)
    return df

def generate_data(df, plate, name, h=20, m=0):
    
    start_idx = 3
    mid_idx = h * 3 + start_idx + 1 - m
    end_idx = h * 3 + mid_idx + 1 - m

    fluo = (df.iloc[:, start_idx:mid_idx]).astype(float)
    od = (df.iloc[:, mid_idx:end_idx]).astype(float)
    fluo_half = (df.iloc[:, end_idx:]).astype(float)
    
    fluo = pd.concat([name, fluo], axis=1)
    od = pd.concat([name, od], axis=1)
    fluo_half = pd.concat([name, fluo_half], axis=1)
    
    fluo = transpose_data(fluo)
    od = transpose_data(od)
    fluo_half = transpose_data(fluo_half)
    
    return fluo, od, fluo_half

### Marionette Strain

In [5]:
def read_plate_map(data):
    
    con = data.iloc[:,:2].reset_index()
    mar = data.iloc[:,2:].reset_index().melt(id_vars=['group'])
    mar['variable'] = mar['variable'].apply(lambda x: "{:02d}".format(int(x)))
    mar['well'] = mar['group'] + mar['variable']
    mar.drop(['group', 'variable'], axis=1, inplace=True)
    mar.rename(columns={'value':'id'}, inplace=True)
    return con, mar

con, mar = read_plate_map(pd.read_csv('datasets/mario_map.csv', index_col=['group']))

In [6]:
def read_plate_data(df):
    
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    return df.reset_index(drop=True)

data = read_plate_data(pd.read_csv('datasets/marrionette.csv'))

In [7]:
merged = pd.merge(data['well'], mar, on='well', how='left')
merged = pd.merge(merged, constructs, on='id', how='left')
name = merged['short_name'].dropna().reset_index(drop=True) #just to make sure there is no null and indexing is correct

In [34]:
merged

,well,id,short_name,full_name
0,A01,A18,e11x32STPhoRadA,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
1,A02,A29,e15x32NpuSspS2,3K3-P15*-gfp-B15-P(ara)-sECF15*-SP2-P(Cym)-32-...
2,A03,A76,e16x33NrdA2,3K3-P16*-gfp-B15-P(ara)-sECF16*-SP31-P(Cym)-33...
3,A04,A109,e20x32gp411,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
4,A05,A195,e32x30SspGyrB,3K3-P32*-gfp-B15-P(ara)-sECF32*-SP8-P(Cym)-30-...
...,...,...,...,...
91,H08,A294,e41x32NrdJ1,3K3-P41*-gfp-B15-P(ara)-sECF41*-SP5-P(Cym)-32-...
92,H09,A323,e42x32STIMPDH1,3K3-P42*-gfp-B15-P(ara)-sECF42*-SP6-P(Cym)-32-...
93,H10,C-,negative_control,NaN
94,H11,C+,positive_control,NaN


In [8]:
mar_fluo, mar_od, mar_fluo_half = generate_data(data, mar, name, 24)
gates = name.unique().tolist()

In [9]:
def reformat_df(data):
    
    top10_wrapper = pd.DataFrame()
    mario_wrapper = pd.DataFrame()

    for gate in gates:

        top10 = data[gate].iloc[:,:4]
        top10.columns = [gate + '_00', gate + '_10', gate + '_01', gate + '_11']
        top10_wrapper = pd.concat([top10_wrapper, top10], axis=1)

        mario = data[gate].iloc[:,4:]
        mario.columns = [gate + '_00', gate + '_10', gate + '_01', gate + '_11']
        mario_wrapper = pd.concat([mario_wrapper, mario], axis=1)
        
    return top10_wrapper, mario_wrapper

In [10]:
top10_fluo, mario_fluo = reformat_df(mar_fluo)
#top10_od, mario_od = reformat_df(mar_od)

#mario_fluo.reset_index().to_csv('datasets/marionette_fluo_half.csv', index=False)
#mario_od.reset_index().to_csv('datasets/marionette_od.csv', index=False)

### Induction Matrix

In [3]:
fluos = []
ods = []

In [4]:
gates = ['e11x32STPhoRadA', 'e15x32NpuSspS2', 'e16x33NrdA2', 'e20x32gp411', 'e32x30SspGyrB', 'e34x30MjaKlbA',
         'e38x32gp418', 'e41x32NrdJ1', 'e42x32STIMPDH1', 'positive_control', 'negative_control', 'blank']

In [5]:
def read_plate_map(raw_data):
    
    data = raw_data.melt(id_vars=['group'])
    data.rename(columns={'value':'id'}, inplace=True)
    data['variable'] = data['variable'].astype(int)
    data['well'] = data['group'] + data['variable'].apply(lambda x: "{:02d}".format(x))
    data['group'] = data['group'].apply(lambda x: ord(x)) #convert alphabet into number
    for i in range(65, 71):
        data.loc[data['group']==i, 'ara'] = i-65
    for i in range(1, 7):
        data.loc[data['variable'].isin([i, i+6]), 'cuma'] = i-1
    return data[['id', 'well', 'ara', 'cuma']]

plate_map = read_plate_map(pd.read_csv('datasets/induction_matrix/induction_map.csv'))

In [23]:
def read_plate_data(df, plate_map, gate, plate_number):
    
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df = df.reset_index(drop=True)
    sample_map = {
        'E1': gate,
        #'E2': gate[1],
        'P_3K3': 'positive_control_3K3',
        'P_4A3': 'positive_control_4AE',
        'N_3K3': 'negative_control_3K3',
        'N_4A3': 'negative_control_4AE',
        'B': 'blank'
    }
    df = pd.merge(df, plate_map, on='well', how='left').dropna(subset=['id']).reset_index(drop=True)
    df.loc[df['id'].isin(['P_3K3', 'P_4AE', 'N_3K3', 'N_4AE', 'B']), 'cuma'] = np.NaN
    df['short_name'] = df['id'].map(sample_map)
    sel = ['id', 'short_name', 'cuma', 'ara']
    df[['cuma', 'ara']] = df[['cuma', 'ara']].fillna(9).astype(int).astype(str)
    df.loc[df['ara']!='9', 'short_name'] = df['short_name'] + '_' + df['cuma'] + df['ara']
    df.loc[df['ara']=='9', 'short_name'] = df['short_name'] + '_' + str(plate_number)
    
    return df['short_name'], df.drop(sel, axis=1)

name, data = read_plate_data(pd.read_csv('datasets/induction_matrix/plate5.csv'), plate_map, gates[8], 5)

In [24]:
name = name.fillna('unknown_samples')

In [7]:
def transpose_data(df):
    
    df.set_index('short_name', inplace=True)
    df = df.transpose().reset_index()
    df['time'] = df['index'].apply(parse_minutes)
    df = df.set_index('time')
    df = df.drop('index', axis=1)
    return df

def generate_data_simple(df, name, h=20, m=0):
    
    start_idx = 3
    mid_idx = h * 3 + start_idx + 1 - m
    end_idx = h * 3 + mid_idx + 1 - m

    fluo = (df.iloc[:, start_idx:mid_idx]).astype(float)
    od = (df.iloc[:, mid_idx:end_idx]).astype(float)
    fluo_half = (df.iloc[:, end_idx:]).astype(float)
    
    fluo = pd.concat([name, fluo], axis=1)
    od = pd.concat([name, od], axis=1)
    fluo_half = pd.concat([name, fluo_half], axis=1)
    
    fluo = transpose_data(fluo)
    od = transpose_data(od)
    fluo_half = transpose_data(fluo_half)
    
    return fluo, od, fluo_half

In [25]:
im_fluo, im_od, im_fluo_half = generate_data_simple(data, name, 24)

In [26]:
im_fluo

short_name,e42x32STIMPDH1_00,e42x32STIMPDH1_10,e42x32STIMPDH1_20,e42x32STIMPDH1_30,e42x32STIMPDH1_40,e42x32STIMPDH1_50,unknown_samples,unknown_samples,unknown_samples,e42x32STIMPDH1_01,e42x32STIMPDH1_11,e42x32STIMPDH1_21,e42x32STIMPDH1_31,e42x32STIMPDH1_41,e42x32STIMPDH1_51,unknown_samples,unknown_samples,unknown_samples,e42x32STIMPDH1_02,e42x32STIMPDH1_12,e42x32STIMPDH1_22,e42x32STIMPDH1_32,e42x32STIMPDH1_42,e42x32STIMPDH1_52,unknown_samples,unknown_samples,unknown_samples,e42x32STIMPDH1_03,e42x32STIMPDH1_13,e42x32STIMPDH1_23,e42x32STIMPDH1_33,e42x32STIMPDH1_43,e42x32STIMPDH1_53,unknown_samples,unknown_samples,unknown_samples,e42x32STIMPDH1_04,e42x32STIMPDH1_14,e42x32STIMPDH1_24,e42x32STIMPDH1_34,e42x32STIMPDH1_44,e42x32STIMPDH1_54,unknown_samples,unknown_samples,unknown_samples,e42x32STIMPDH1_05,e42x32STIMPDH1_15,e42x32STIMPDH1_25,e42x32STIMPDH1_35,e42x32STIMPDH1_45,e42x32STIMPDH1_55,unknown_samples,unknown_samples,unknown_samples,negative_control_3K3_5,negative_control_3K3_5,negative_control_3K3_5,positive_control_3K3_5,positive_control_3K3_5,positive_control_3K3_5,negative_control_4AE_5,negative_control_4AE_5,negative_control_4AE_5,positive_control_4AE_5,positive_control_4AE_5,positive_control_4AE_5,blank_5,blank_5,blank_5,blank_5,blank_5,blank_5
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,37.000,37.000,67.000,63.000,64.000,94.000,116.000,71.000,NaN,24.000,74.000,72.000,70.000,96.000,99.000,80.000,108.000,NaN,57.000,71.000,86.000,75.000,101.000,101.000,124.000,103.000,NaN,28.000,75.000,86.000,84.000,113.000,121.000,125.000,91.000,NaN,84.000,81.000,62.000,89.000,94.000,106.000,115.000,94.000,NaN,22.000,38.000,94.000,66.000,64.000,85.000,68.000,84.000,NaN,42.000,62.000,69.000,5234.000,5420.000,5881.000,75.333,95.333,78.333,1263.333,871.333,912.333,NaN,NaN,NaN,NaN,NaN,NaN
20,35.333,34.333,61.333,69.333,60.333,71.333,77.333,63.333,NaN,26.333,57.333,76.333,42.333,86.333,80.333,58.333,80.333,NaN,37.333,67.333,64.333,85.333,83.333,94.333,97.333,81.333,NaN,27.333,50.333,92.333,108.333,102.333,104.333,105.333,80.333,NaN,55.333,66.333,51.333,81.333,105.333,93.333,115.333,94.333,NaN,44.333,58.333,77.333,99.333,47.333,62.333,73.333,94.333,NaN,91.333,81.333,107.333,4258.333,4153.333,4171.333,82.667,81.667,61.667,773.667,763.667,765.667,NaN,NaN,NaN,NaN,NaN,NaN
40,39.333,35.333,75.333,57.333,52.333,81.333,63.333,90.333,NaN,34.333,52.333,42.333,63.333,72.333,70.333,63.333,65.333,NaN,54.333,61.333,79.333,69.333,85.333,75.333,80.333,93.333,NaN,38.333,40.333,69.333,76.333,88.333,71.333,97.333,61.333,NaN,94.333,49.333,54.333,73.333,89.333,67.333,68.333,74.333,NaN,24.333,62.333,60.333,71.333,52.333,44.333,79.333,61.333,NaN,67.333,63.333,95.333,4564.333,4358.333,4402.333,88.667,74.667,69.667,1033.667,972.667,1025.667,NaN,NaN,NaN,NaN,NaN,NaN
60,60.333,36.333,82.333,58.333,62.333,75.333,75.333,79.333,NaN,41.333,49.333,60.333,73.333,72.333,106.333,70.333,77.333,NaN,38.333,53.333,77.333,71.333,97.333,85.333,87.333,81.333,NaN,48.333,73.333,87.333,78.333,86.333,126.333,68.333,74.333,NaN,62.333,51.333,57.333,76.333,84.333,57.333,78.333,68.333,NaN,69.333,56.333,54.333,61.333,91.333,95.333,63.333,57.333,NaN,94.333,102.333,108.333,4839.333,4662.333,4709.333,78.000,93.000,52.000,1368.000,1304.000,1393.000,NaN,NaN,NaN,NaN,NaN,NaN
80,53.000,53.000,68.000,63.000,78.000,74.000,87.000,79.000,NaN,25.000,61.000,66.000,68.000,86.000,92.000,83.000,73.000,NaN,77.000,74.000,93.000,82.000,71.000,83.000,92.000,83.000,NaN,57.000,62.000,90.000,113.000,115.000,115.000,78.000,71.000,NaN,65.000,75.000,63.000,90.000,84.000,77.000,101.000,65.000,NaN,52.000,64.000,106.000,89.000,66.000,71.000,73.000,78.000,NaN,77.000,103.000,103.000,5319.000,5108.000,5140.000,103.000,89.000,109.000,1802.000,1729.000,1768.000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [27]:
fluos.append(im_fluo)
ods.append(im_od)

In [28]:
len(fluos)

5

In [29]:
fluos2 = pd.concat(fluos, axis=1)
ods2 = pd.concat(ods, axis=1)

In [30]:
fluos2.to_csv('datasets/induction_matrix/induction_fluo.csv')
ods2.to_csv('datasets/induction_matrix/induction_od.csv')

### ALL GATES

### Data from plate reader 1

In [5]:
def cleanse_plate(plate):
    plate = plate.melt(id_vars=['Unnamed: 0'])
    plate['value'] = plate['value'].apply(lambda x: x.split('.')[0])
    plate['variable'] = plate['variable'].apply(lambda x: "{:02d}".format(int(x)))
    plate['variable'] = plate['Unnamed: 0'] + plate['variable']
    plate.drop('Unnamed: 0', axis=1, inplace=True)
    return plate.reset_index(drop=True).rename(columns={'variable': 'Well', 'value': 'code_name'})

plate1 = cleanse_plate(pd.read_csv('datasets/plate1_map.csv'))
plate1

,Well,code_name
0,A01,A5
1,B01,A37
2,C01,A59
3,D01,A79
4,E01,A95
...,...,...
91,D12,A94
92,E12,A105
93,F12,A121
94,G12,A137


In [6]:
df11 = read_plate_data('and_gate_11') # up to 20h
print(df11.shape)
df12 = read_plate_data('and_gate_12') # up to 16h
print(df12.shape)
df13 = read_plate_data('and_gate_13') # up to 16h
print(df13.shape)

AttributeError: 'str' object has no attribute 'iloc'

In [ ]:
merged = pd.merge(df11['Well'], plate1, on='Well', how='left')
merged = pd.merge(merged, naming_map, on='code_name', how='left')
name1 = merged['short_name']

In [ ]:
fluo11, od11, bulk_fluo11, fluo_half11, bulk_fluo_half11 = generate_data(df11, plate1, name1, 20)
fluo12, od12, bulk_fluo12, fluo_half12, bulk_fluo_half12 = generate_data(df12, plate1, name1, 16)
fluo13, od13, bulk_fluo13, fluo_half13, bulk_fluo_half13 = generate_data(df13, plate1, name1, 16)

In [ ]:
def plot_all(data, num_row, num_col):
    f, axs = plt.subplots(num_row, num_col, sharex=True, sharey=False, figsize=(14, num_row*2))
    axr = axs.ravel()
    for i, ax in tqdm(enumerate(axr)):
        if i < data[0].shape[1]:
            for d in data:
                ax.plot(d.index/60, d.iloc[:, i])
            ax.set_title(data[0].columns[i])
            ax.set_xlabel('Time (h)')
        else:
            ax.set_visible(False)
    plt.tight_layout()
    sns.despine()
    
#plot bulk fluorescence data
plot_all([bulk_fluo_half11, bulk_fluo_half12, bulk_fluo_half13], 20, 5)

In [ ]:
plot_all([od11, od12, od13], 20, 5)

In [ ]:
bulk_fluo1 = (bulk_fluo11 + bulk_fluo12 + bulk_fluo13) / 3
fluo1 = (fluo11 + fluo12 + fluo13) / 3
od1 = (od11 + od12 + od13) / 3
fluo_half1 = (fluo_half11 + fluo_half12 + fluo_half13) / 3
bulk_fluo_half1 = (bulk_fluo_half11 + bulk_fluo_half12 + bulk_fluo_half13) / 3

In [ ]:
bulk_fluo1.dropna().to_csv('datasets/bulk_fluo_plate_1_triplicate.csv')
fluo1.dropna().to_csv('datasets/fluo_plate_1_triplicate.csv')
od1.dropna().to_csv('datasets/od_plate_1_triplicate.csv')
bulk_fluo_half1.dropna().to_csv('datasets/bulk_fluo_half_plate_1_triplicate.csv')
fluo_half1.dropna().to_csv('datasets/fluo_half_plate_1_triplicate.csv')

In [ ]:
bulk_fluo11.dropna().to_csv('datasets/bulk_fluo_plate_1_single.csv')
fluo11.dropna().to_csv('datasets/fluo_plate_1_single.csv')
od11.dropna().to_csv('datasets/od_plate_1_single.csv')
bulk_fluo_half11.dropna().to_csv('datasets/bulk_fluo_half_plate_1_single.csv')
fluo_half11.dropna().to_csv('datasets/fluo_half_plate_1_single.csv')

### Data from plate reader 2

In [ ]:
plate2 = cleanse_plate(pd.read_csv('datasets/plate2_map.csv'))
df21 = read_plate_data('and_gate_21') # up to 20h
print(df21.shape)
df22 = read_plate_data('and_gate_22') # up to 16h
print(df22.shape)
df23 = read_plate_data('and_gate_23') # up to 16h
print(df23.shape)

In [ ]:
merged = pd.merge(df21['Well'], plate2, on='Well', how='left')
merged = pd.merge(merged, naming_map, on='code_name', how='left')
name2 = merged['short_name']

In [ ]:
fluo21, od21, bulk_fluo21, fluo_half21, bulk_fluo_half21 = generate_data(df21, plate2, name2, 21, 2)
fluo22, od22, bulk_fluo22, fluo_half22, bulk_fluo_half22 = generate_data(df22, plate2, name2, 22, 0)
fluo23, od23, bulk_fluo23, fluo_half23, bulk_fluo_half23 = generate_data(df23, plate2, name2, 22, 1)

In [ ]:
plot_all([bulk_fluo_half21, bulk_fluo_half22, bulk_fluo_half23], 20, 5)

In [ ]:
plot_all([od21, od22, od23], 20, 5)

In [ ]:
bulk_fluo2 = (bulk_fluo21 + bulk_fluo22 + bulk_fluo23) / 3
fluo2 = (fluo21 + fluo22 + fluo23) / 3
od2 = (od21 + od22 + od23) / 3
fluo_half2 = (fluo_half21 + fluo_half22 + fluo_half23) / 3
bulk_fluo_half2 = (bulk_fluo_half21 + bulk_fluo_half22 + bulk_fluo_half23) / 3

In [ ]:
bulk_fluo2.to_csv('datasets/bulk_fluo_plate_2_triplicate.csv')
fluo2.to_csv('datasets/fluo_plate_2_triplicate.csv')
fluo_half2.to_csv('datasets/fluo_half_plate_2_triplicate.csv')
bulk_fluo_half2.to_csv('datasets/bulk_fluo_half_plate_2_triplicate.csv')
od2.to_csv('datasets/od_plate_2_triplicate.csv')

In [ ]:
bulk_fluo21.to_csv('datasets/bulk_fluo_plate_2_single.csv')
fluo21.to_csv('datasets/fluo_plate_2_single.csv')
fluo_half21.to_csv('datasets/fluo_half_plate_2_single.csv')
bulk_fluo_half21.to_csv('datasets/bulk_fluo_half_plate_2_single.csv')
od21.to_csv('datasets/od_plate_2_single.csv')

### Induction Matrix

In [ ]:
induction = read_plate_data('induction') # up to 20h
print(induction.shape)

In [ ]:
induction_plate = {
    'A01': 'A18', 'A02': 'A18', 'A03': 'A18', 
    'B01': 'A18', 'B02': 'A18', 'B03': 'A18', 
    'C01': 'A18', 'C02': 'A18', 'C03': 'A18', 
    'D01': 'A195', 'D02': 'A195', 'D03': 'A195', 
    'E01': 'A195', 'E02': 'A195', 'E03': 'A195', 
    'F01': 'A195', 'F02': 'A195', 'F03': 'A195', 
    'A04': 'A29', 'A05': 'A29', 'A06': 'A29', 
    'B04': 'A29', 'B05': 'A29', 'B06': 'A29', 
    'C04': 'A29', 'C05': 'A29', 'C06': 'A29', 
    'D04': 'A259', 'D05': 'A259', 'D06': 'A259', 
    'E04': 'A259', 'E05': 'A259', 'E06': 'A259', 
    'F04': 'A259', 'F05': 'A259', 'F06': 'A259', 
    'A07': 'A76', 'A08': 'A76', 'A09': 'A76', 
    'B07': 'A76', 'B08': 'A76', 'B09': 'A76', 
    'C07': 'A76', 'C08': 'A76', 'C09': 'A76', 
    'D07': 'A267', 'D08': 'A267', 'D09': 'A267', 
    'E07': 'A267', 'E08': 'A267', 'E09': 'A267', 
    'F07': 'A267', 'F08': 'A267', 'F09': 'A267', 
    'A10': 'A109', 'A11': 'A109', 'A12': 'A109', 
    'B10': 'A109', 'B11': 'A109', 'B12': 'A109', 
    'C10': 'A109', 'C11': 'A109', 'C12': 'A109', 
    'D10': 'A294', 'D11': 'A294', 'D12': 'A294', 
    'E10': 'A294', 'E11': 'A294', 'E12': 'A294', 
    'F10': 'A294', 'F11': 'A294', 'F12': 'A294', 
    'G01': 'A323', 'G02': 'A323', 'G03': 'A323', 
    'G04': 'A323', 'G05': 'A323', 'G06': 'A323', 
    'H01': 'A323', 'H02': 'A323', 'H03': 'A323', 
}
induction_row = {
    'A': 0, 'B': 1, 'C': 2, 'D': 0, 'E': 1, 'F': 2,
    'G': 0, 'H': 1 #column g needs to be manually adjusted later
}
induction_col = {
    '01': 0, '02': 1, '03': 2, '04': 0, '05': 1, '06': 2, 
    '07': 0, '08': 1, '09': 2, '10': 0, '11': 1, '12': 2 
}

In [ ]:
index = induction.iloc[:,:3]
index['code_name'] = index['Well'].map(induction_plate)
col_idx = index['Well'].str[1:]
row_idx = index['Well'].str[:1]
index['ind1_lvl'] = row_idx.map(induction_row)
index['ind2_lvl'] = col_idx.map(induction_col)
index.loc[index['Well'].isin(['G04', 'G05', 'G06']), 'ind1_lvl'] = 2
#index = index.dropna()
index.loc[index['code_name'].isnull(), 'code_name'] = 'control'
name = pd.merge(index[['code_name', 'ind1_lvl', 'ind2_lvl']], naming_map[['code_name', 'short_name']], on='code_name', how='left')
name

In [ ]:
index[index['code_name']=='A323']

In [ ]:
def parse_minutes(x):
    spl = x.split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

def transpose_data(df_raw):
    df = df_raw.copy()
    df.set_index('short_name', inplace=True)
    #df = df.transpose().reset_index()
    #df['time'] = df['index'].apply(parse_minutes)
    #df = df.set_index('time')
    #df = df.drop('index', axis=1)
    return df

def generate_data_induce(df, h=24):
    
    start_idx = 3
    mid_idx = h * 3 + start_idx
    fluo = (df.iloc[:, start_idx:mid_idx]).astype(float)
    od = (df.iloc[:, mid_idx:]).astype(float)
    bulk_fluo = fluo * od
    
    fluo = pd.concat([name, fluo], axis=1)
    od = pd.concat([name, od], axis=1)
    bulk_fluo = pd.concat([name, bulk_fluo], axis=1)
    
    #bulk_fluo = transpose_data(bulk_fluo)
    #fluo = transpose_data(fluo)
    #od = transpose_data(od)
    
    return fluo, od, bulk_fluo

fluo, od, bulk_fluo = generate_data_induce(induction)
bulk_fluo

In [ ]:
bulk_fluo[bulk_fluo['short_name']=='e11x32STPhoRadA']

In [ ]:
bulk_fluo.to_csv('datasets/bulk_fluo_induction.csv', index=False)
fluo.to_csv('datasets/fluo_induction.csv', index=False)
od.to_csv('datasets/od_induction.csv', index=False)

### XOR Gate Top 10

In [ ]:
def read_plate_map(df):
    data = pd.read_csv('datasets/xor_map.csv', index_col=['Group'])
    con = data.iloc[:,:2].reset_index()
    xor = data.iloc[:,2:]
    xor = xor.reset_index().melt(id_vars=['Group'])
    xor['variable'] = xor['variable'].apply(lambda x: "{:02d}".format(int(x)))
    xor['Well'] = xor['Group'] + xor['variable']
    xor.drop(['Group', 'variable'], axis=1, inplace=True)
    xor.rename(columns={'value':'code_name'}, inplace=True)
    return con, xor

con, xor = read_plate_map(pd.read_csv('datasets/xor_map.csv'))

In [ ]:
def read_plate_data(df):
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    #df.dropna(inplace=True)
    return df.reset_index(drop=True)

data = read_plate_data(pd.read_csv('datasets/xor_gate.csv'))
data.head()

In [ ]:
merged = pd.merge(data['Well'], xor, on='Well', how='left')
#merged = pd.merge(merged, naming_map, on='code_name', how='left')
merged.rename(columns={'code_name': 'short_name'}, inplace=True)
name = merged['short_name'].dropna().reset_index(drop=True)

In [ ]:
fluox, odx, fluo_halfx = generate_data(data, xor, name, 24)

In [ ]:
cols = ['ECF20/33', 'ECF11/15']
fluo_xor = pd.DataFrame()
od_xor = pd.DataFrame()
for c in cols:
    temp = pd.concat([fluox[c].iloc[:,i] for i in range(0, 16, 4)], axis=1)
    temp.columns = [c + '_' + "{:02b}".format(int(i)) for i in range(4)]
    fluo_xor = pd.concat([fluo_xor, temp], axis=1)
    
    temp = pd.concat([odx[c].iloc[:,i] for i in range(0, 16, 4)], axis=1)
    temp.columns = [c + '_' + "{:02b}".format(int(i)) for i in range(4)]
    od_xor = pd.concat([od_xor, temp], axis=1)

In [ ]:
fluo_xor.to_csv('datasets/bulk_fluo_xor.csv')
od_xor.to_csv('datasets/od_xor.csv')

### First-round Data

In [ ]:
df = pd.read_csv('datasets/raw.csv')
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.dropna(inplace=True)
print(df.shape)
df.head()

In [ ]:
map_ecf = {'Sample X1': 'e15',
'Sample X2': 'e22',
'Sample X3': 'e32',
'Sample X4': 'e33',
'Sample X5': 'e34',
'Sample X6': 'e41',
'Sample X7': 'e42',
'Sample X8': 'e15',
'Sample X9': 'e22',
'Sample X10': 'e38',
'Sample X11': 'e16',
'Sample X12': 'e33',
'Sample X13': 'e15',
'Sample X14': 'e16',
'Sample X15': 'e17',
'Sample X16': 'e20',
'Sample X17': 'e22',
'Sample X18': 'e26',
'Sample X19': 'e32',
'Sample X20': 'e33',
'Sample X21': 'e34'}
map_int = {'Sample X1': 'SspGyrB',
'Sample X2': 'SspGyrB',
'Sample X3': 'SspGyrB',
'Sample X4': 'SspGyrB',
'Sample X5': 'SspGyrB',
'Sample X6': 'SspGyrB',
'Sample X7': 'SspGyrB',
'Sample X8': 'TerThyXS2',
'Sample X9': 'TerThyXS2',
'Sample X10': 'TerThyXS2',
'Sample X11': 'TerThyXS1',
'Sample X12': 'TerThyXS1',
'Sample X13': 'STPhoRadA',
'Sample X14': 'STPhoRadA',
'Sample X15': 'STPhoRadA',
'Sample X16': 'STPhoRadA',
'Sample X17': 'STPhoRadA',
'Sample X18': 'STPhoRadA',
'Sample X19': 'STPhoRadA',
'Sample X20': 'STPhoRadA',
'Sample X21': 'STPhoRadA'}

In [ ]:
df['ECF'] = df['Content'].map(map_ecf)
df['Intein'] = df['Content'].map(map_int)
df.loc[df['Group'].isin(['A', 'E']), 'Ara'] = 0
df.loc[df['Group'].isin(['A', 'E']), 'Cuma'] = 0
df.loc[df['Group'].isin(['B', 'F']), 'Ara'] = 1
df.loc[df['Group'].isin(['B', 'F']), 'Cuma'] = 0
df.loc[df['Group'].isin(['C', 'G']), 'Ara'] = 0
df.loc[df['Group'].isin(['C', 'G']), 'Cuma'] = 1
df.loc[df['Group'].isin(['D', 'H']), 'Ara'] = 1
df.loc[df['Group'].isin(['D', 'H']), 'Cuma'] = 1
df = pd.concat([df[['ECF', 'Intein', 'Ara', 'Cuma']],
                 df.drop(['ECF', 'Intein', 'Ara', 'Cuma', 'Well', 'Content', 'Group'], axis=1)],
                 axis=1)
df.head()

In [ ]:
cols = list(range(0,77))
fluo = df.iloc[:, cols]
#fluo.to_csv('datasets/fluoOD-all.csv', index=False)
fluo.head()

In [ ]:
cols = list(range(0,4)) + list(range(77,150))
od = df.iloc[:, cols]
#od.to_csv('datasets/OD-all.csv', index=False)
od.head()

In [ ]:
def parse_minutes(x):
    spl = x.split(' ')
    hours = int(spl[0]) * 60
    mins = int(spl[2]) if spl[2] != '' else 0
    return hours + mins

fluo11 = fluo[(fluo['Ara']==1) & (fluo['Cuma']==0)]
fluo11['index'] = fluo11['ECF'] + fluo11['Intein']
fluo11.set_index('index', inplace=True)
fluo11.drop(['ECF', 'Intein', 'Ara', 'Cuma'], axis=1, inplace=True)
fluo11 = fluo11.transpose().reset_index()

fluo11['time'] = fluo11[0].apply(parse_minutes)
fluo11 = fluo11.set_index('time')
fluo11 = fluo11.drop(0, axis=1)
fluo11

In [ ]:
od11 = od[(od['Ara']==1) & (od['Cuma']==0)]
od11['index'] = od11['ECF'] + od11['Intein']
od11.set_index('index', inplace=True)
od11.drop(['ECF', 'Intein', 'Ara', 'Cuma'], axis=1, inplace=True)
od11 = od11.transpose().reset_index()

od11['time'] = od11[0].apply(parse_minutes)
od11 = od11.set_index('time')
od11 = od11.drop(0, axis=1)
od11

In [ ]:
for col in fluo11.columns:
    try:
        fluo11[col] = fluo11[col].astype(float)
    except:
        print(col)
        continue

In [ ]:
for col in od11.columns:
    od11[col] = od11[col].astype(float)

In [ ]:
fluo11.to_csv('datasets/fluo-10.csv')
od11.to_csv('datasets/od-10.csv')
(fluo11 * od11).to_csv('datasets/bulk-fluo-10.csv')

### Sequence to Function

In [2]:
filenames = sorted(os.listdir('datasets/sequences/'))
buffer = {}
for filename in tqdm(filenames):
    gb_file = "datasets/sequences/" + filename
    for gb_record in SeqIO.parse(open(gb_file,"r"), "genbank") :
        buffer[filename[:-3]] = str(gb_record.seq)
df = pd.DataFrame.from_dict(buffer, orient='index').reset_index()
df.columns = ['full_name', 'sequence']

100%|██████████| 355/355 [00:01<00:00, 299.54it/s]


In [13]:
constructs = pd.read_csv('datasets/constructs.csv')
constructs['full_name'] = constructs['full_name'].str.replace("*", "")
df2 = pd.merge(df, constructs, on="full_name", how="left")
df2 = df2.dropna()
df2 = df2[['id', 'short_name', 'full_name', 'sequence']]
fluo1 = pd.read_csv('datasets/bulk_fluo_plate_1_single.csv', index_col='time')
fluo2 = pd.read_csv('datasets/bulk_fluo_plate_2_single.csv', index_col='time')
fluo = pd.concat([fluo1, fluo2], axis=1)
data = fluo.T.reset_index().drop(1220, axis=1)
data.rename(columns={'index': 'short_name'}, inplace=True)
#data.columns = ['short_name', 'fluo_20h']
df_final = pd.merge(df2, data, on='short_name', how='left').dropna()
df_final

<ipython-input-13-31308874845c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  constructs['full_name'] = constructs['full_name'].str.replace("*", "")


,id,short_name,full_name,sequence,0,20,40,60,80,100,120,140,160,180,200,220,240,260,280,300,320,340,360,380,400,420,440,460,480,500,520,540,560,580,600,620,640,660,680,700,720,740,760,780,800,820,840,860,880,900,920,940,960,980,1000,1020,1040,1060,1080,1100,1120,1140,1160,1180,1200
3,A18,e11x32STPhoRadA,3K3-P11-gfp-B15-P(ara)-sECF11-SP11-P(Cym)-32-(...,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...,36.20450,-123.98541,-385.857160,-14.89200,3.82501,36.29976,39.46566,28.59230,67.85850,749.06865,2942.10049,6863.80695,11021.14318,16089.68025,22241.14200,29534.11230,38314.78605,49202.17155,59555.58368,61088.68145,62059.03209,64442.41440,67196.02244,70129.93043,73928.80947,77564.50499,82043.73947,85646.97729,89723.85500,93543.08068,97331.30756,100810.89216,103986.58300,106829.20620,108622.16640,110419.21514,111831.98865,113745.84936,114921.83844,116100.53323,116717.60250,117561.54698,118486.65873,119346.34712,119884.19075,119964.83323,121134.38625,121480.66612,122688.45860,122670.17410,124115.81230,124686.27831,125398.10328,126375.57600,127123.82523,127476.36564,127894.02930,128569.76450,128820.78768,128710.99332,130043.56617
8,A20,e11x32CthTer,3K3-P11-gfp-B15-P(ara)-sECF11-SP20-P(Cym)-32-s...,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...,51.52824,95.13588,118.499997,74.05932,50.82480,52.64940,59.61240,0.65512,54.82233,66.33195,151.60090,684.20970,1316.32160,2068.52935,3202.74828,4710.76221,6785.05600,9090.26140,10290.86160,9736.09182,10035.53655,10143.41400,10543.65148,10740.25224,10937.40800,11134.43520,11427.46448,11682.94050,11915.52648,12171.71538,12350.62737,12350.47840,12321.67200,12373.26314,12402.85670,12346.81446,12229.75104,12222.93624,12171.02982,12316.52715,12174.34145,12099.07110,12097.40847,12234.64840,12032.65388,12093.48928,12068.09307,12015.96475,12115.35360,12149.84628,12295.98666,12178.69755,12274.97172,12319.89560,12288.54405,12352.45500,12331.22751,12476.21264,12395.23875,12598.14375,12587.09244
11,A5,e11x32gp411,3K3-P11-gfp-B15-P(ara)-sECF11-SP3-P(Cym)-32-sg...,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...,-7.95690,-171.07916,-249.729480,16.87728,45.30954,30.79230,79.61772,60.88710,65.83080,291.53670,2152.41516,5120.52805,8786.86000,12921.87360,17844.94215,23332.33972,30056.49241,37710.21940,46762.43781,57480.48417,64585.66530,65248.70129,66003.65772,67650.19720,71332.08320,74048.04000,78639.91971,83247.16400,88693.54368,94841.99100,100280.87764,107331.32146,112134.19200,116777.70920,121694.87901,123891.18140,127191.03408,128935.33804,131723.63145,135518.79240,138493.00465,143246.59232,146306.74110,148192.74918,150926.32571,150817.49936,156097.17054,157081.07820,156762.96516,163127.07928,161485.78236,166272.36240,164888.33519,169559.42370,172330.50614,172859.41938,173275.76822,178951.20912,178986.39836,177490.23384,180732.79836
12,A23,e11x32NrdA2,3K3-P11-gfp-B15-P(ara)-sECF11-SP31-P(Cym)-32-s...,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...,-69.45120,-86.64228,-125.931280,-11.25367,16.56139,13.21575,27.64211,21.98376,43.82400,1452.51838,5883.48155,12803.70130,21272.06772,31481.67870,43703.09129,57868.70760,74315.92278,94729.98744,117415.14980,129090.25404,130911.31918,132368.17644,137820.43577,144681.91872,152750.75196,161360.34489,170053.23544,179274.08742,188083.83100,197337.13000,205729.81342,213732.37340,218902.91556,224463.90964,229802.55396,233871.90792,238832.32331,242905.81796,245958.38226,249963.16025,251381.46069,253194.49937,253268.14902,253404.31744,253358.43116,253301.09157,253117.42500,253170.63792,253173.21990,253145.68980,253084.60244,252975.20364,252938.36835,252970.74102,252927.29363,252881.36649,252688.37308,252843.08984,252666.28668,252778.69654,252783.57335
13,A24,e11x33NrdA2,3K3-P11-gfp-B15-P(ara)-sECF11-SP31-P(Cym)-33-s...,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...,115.31576,131.82396,104.071429,85.78220,66.43464,48.31920,62.65038,44.93046,81.15289,1792.20945,5914.98016,10934.76560,17820.27776,26858.11456,37730.41196,51065.27846,66882.85384,81737.64467,84534.49845,8

In [14]:
df_final.isnull().sum()

id            0
short_name    0
full_name     0
sequence      0
0             0
             ..
1120          0
1140          0
1160          0
1180          0
1200          0
Length: 65, dtype: int64

In [15]:
df_final.to_csv('datasets/sequence_data.csv')